## Inherit the CropImage Class and add shapelet related methods

In [1]:
import pandas as pd
# Load 'Prob_Image' class that was defined in ProbImage_CLass.ipynb
%run ProbImage_Class.ipynb

class Shapelet_Image(Prob_Image):
    shapelet_image = []         # image
    current_windowsize = None
    
    def shapelet_2d(self, w=3, mask=None,out_type=np.uint16,fill_val=999):
        
        # mask can be a slope-threshold based mask 
        # made from ProbImage_Class.mask_of_slope()
        # e.g., test1.mask_of_slope(-0.03, usenan=1)
        
        row = self.crop_stack.shape[1]
        col = self.crop_stack.shape[2]
        band_num = self.crop_stack.shape[0]

        image_cropprob_flat = self.crop_stack.transpose(2,1,0).reshape(row*col,band_num)
          
        ts_2d = image_cropprob_flat
        # get the length of the timeseires
        l = ts_2d.shape[1]

        # construct candidate_shapelet list and candidate_nonshapelet list
        candidate_S_list = []
        candidate_N_list = []
        i = 0
        while i+(w-1) < l:
            #candidate_S_list.append(list(range(i, i+w)))
            #candidate_N_list.append([list(range(0, i)),list(range(i+w, l-w+2))])

            S_ind = [i, i+w-1]  # a moving window size =3

            # if left non-shapelet segment exists, return its index
            if i > 0:
                N1_ind = [0, i-1]
            # else, return empty list
            else:
                N1_ind = []

            # if right non-shapelet exists, return index
            if i+w < l:
                N2_ind = [i+w, l-1]
            # else, return empty index
            else:
                N2_ind = []

            candidate_S_list.append(S_ind)
            candidate_N_list.append([N1_ind, N2_ind])
            i += 1

        # calculate GAP
        GAP_list = []

        # T test (T test 1: S and N1; T test 2: S and N2)
        t_val1_list = [] 
        t_val2_list = [] 


        for i in range(0,len(candidate_S_list)):
            S = ts_2d[:, candidate_S_list[i][0]:candidate_S_list[i][1]+1]   # for all rows, slice on ts based on candidate_s list
            S_sd = np.std(S, axis=1)
            S_mean = np.mean(S, axis=1)
            N_sd_list = []
            # loop to inlucde N1 and N2
            for N_ind in candidate_N_list[i]:
                if N_ind != []:
                    N = ts_2d[:, N_ind[0]:N_ind[1]+1]
                    N_sd_list.append(np.std(N, axis=1))                
            # calculate GAP
            N_mean_sd = np.mean(N_sd_list, axis=0) # the mean of two sd
            GAP = S_sd - N_mean_sd
            GAP_list.append(GAP.tolist())


        GAP_list = np.array(GAP_list).transpose(1,0)

        GAP_list_sorted = np.argsort(GAP_list, axis=1) # return index of the sorted list

        S_selected = GAP_list_sorted[:,-1]  # the start time point for the selected Shapelet

        print('GAP done')

        print(len(S_selected))

        # S_selected is also the index for candidate_S_list
        S_selected_list = []
        j=0
        for i in S_selected:
            #print(test_ts[j,i:i+3])
            S_selected_list.append(ts_2d[j,i:i+w].tolist())
            j+=1
        S_selected_list = np.array(S_selected_list)



    #     # N1.
    #     j=0
    #     N1_selected_list = []
    #     for i in S_selected:
    #         N_temp = np.full((1,l-w),np.nan)             # creat a 1d nan array, use the potential max length of N, should be l-w
    #         N_temp[:1,:i] =  ts_2d[j, 0:i].tolist()  # populate na array with N, may has a nan tail if len(N) < l-w
    #         N1_selected_list.append(N_temp.tolist()[0])

    #     N1_selected_list = np.array(N1_selected_list)    

    #     # N2.
    #     j=0
    #     N2_selected_list = []
    #     for i in S_selected:
    #         N_temp = np.full((1,l-w),np.nan)
    #         N_temp[:1,:(l-i-w)] =  ts_2d[j, i+w:].tolist()
    #         N2_selected_list.append(N_temp.tolist()[0])

    #     N2_selected_list = np.array(N2_selected_list)    




    #     # N1 and N2 together.
    #     j=0
    #     N1_selected_list = []
    #     N2_selected_list = []
    #     for i in S_selected:
    #         N1_temp = np.full((1,l-w),np.nan)             # creat a 1d nan array, use the potential max length of N, should be l-w
    #         N1_temp[:1,:i] =  ts_2d[j, 0:i].tolist()  # populate na array with N, may has a nan tail if len(N) < l-w
    #         N1_selected_list.append(N1_temp.tolist()[0])

    #         N2_temp = np.full((1,l-w),np.nan)
    #         N2_temp[:1,:(l-i-w)] =  ts_2d[j, i+w:].tolist()
    #         N2_selected_list.append(N2_temp.tolist()[0])

    #     N1_selected_list = np.array(N1_selected_list)  
    #     N2_selected_list = np.array(N2_selected_list) 





        # N1 and N2 together use pandas to coarse each row to have same length, faster than padding nan for each row.
        j=0
        N1_selected_list = []
        N2_selected_list = []
        for i in S_selected:
            N1_selected_list.append(ts_2d[j, 0:i].tolist())
            N2_selected_list.append(ts_2d[j, i+w:].tolist())



        N1_selected_list = pd.DataFrame(N1_selected_list).to_numpy()
        N2_selected_list = pd.DataFrame(N2_selected_list).to_numpy()




        # convert inconsistent length lists to pandas then to numpy
        #pd.DataFrame(a).to_numpy()


        def t_test_2d(S,N):
            # S is the shapelet_2d in m*w, where m is the number of rows (pixels) and l is the length of window 
            # N is the Non-shapelet, either on the left or right of the shapelet.  The shape is m*v, which v varies from 0 to l-w

            S_mean = np.mean(S, axis=1)
            S_var = np.var(S, axis=1)                    
            N_mean = np.nanmean(N, axis=1)
            N_var = np.nanvar(N, axis=1)

            # count number of non-nan value for each row in N
            n_count = np.count_nonzero(~np.isnan(N), axis=1).astype('float')
            n_count[n_count==0]=np.nan

           # because we are not sure  N1_selected_mean -  S_selected_mean is positive or negative, we take abs
            numerator = np.absolute(N_mean - S_mean)
            denominator1_upper = np.absolute((n_count-1)*N_var**2 + (w-1)*S_var**2) 
            denominator1_lower = (n_count+w-2)
            denominator1 = (denominator1_upper/denominator1_lower)**0.5  
            denominator2 = ((1/(n_count)) +1/w)**0.5
            t =  numerator/(denominator1*denominator2) 
            return(t)


        print('Start t score calculation')

        t1_score = t_test_2d(S_selected_list,N1_selected_list)
        t2_score = t_test_2d(S_selected_list,N2_selected_list)
        
        result = S_selected.reshape(col, row).transpose(1,0)
        

        self.current_windowsize = w
        
        if mask is None:
            print('The mask is none')
            self.shapelet_image = result.astype(np.uint16)         # image
        else:
            print('Using mask')           
            self.shapelet_image = ((result*mask).filled(fill_val)).astype(out_type)        # image
            
        print('before return')
        return self.shapelet_image#result




In [ ]:
#test1 = Shapelet_Image('/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/BULCU-all_2016_backward/grid_1766_BULCU.tif')

# from matplotlib import pyplot
# fig, axs = pyplot.subplots(1, 2,figsize = [20,10]) # row, col
# subplt_title_size = 20
# pyplot.subplots_adjust(bottom=0.1, top=1, hspace=0)
# pyplot.tight_layout()
# #fig.set_size_inches(20.5, 12.5)
# axs[0].imshow(test1.shapelet_2d(w=3, mask=test1.mask_of_slope(-0.03, usenan=1)), cmap='RdYlBu')
# axs[1].imshow(test1.shapelet_2d(w=3, mask=test1.mask_of_slope(-0.02, usenan=1)), cmap='RdYlBu')

In [3]:
# test1 = Shapelet_Image('/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/BULCU-all_2016_backward/grid_1766_BULCU.tif')
# test1.shapelet_2d(w=3, mask=test1.mask_of_slope(-0.02, usenan=1))

# out_fold = 'masked_shapelet002'
# grid_id = '1766-002'
# test1.generic_export(image_type='shapelet', 
#                      out_path='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/'+out_fold,
#                      out_name='shapelet_image_masked_'+grid_id+'.tif')

GAP done
640048
Start t score calculation


/tmp/ipykernel_84596/3257797922.py:167: RuntimeWarning: Mean of empty slice
  N_mean = np.nanmean(N, axis=1)
/tmp/ipykernel_84596/3257797922.py:168: RuntimeWarning: Degrees of freedom <= 0 for slice.
  N_var = np.nanvar(N, axis=1)
/tmp/ipykernel_84596/3257797922.py:180: RuntimeWarning: invalid value encountered in true_divide
  t =  numerator/(denominator1*denominator2)


Using mask
before return
